#### Arithmetic Expressions with Constants in P0 (WASM)

In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Analyze the generated code of the following program:

In [2]:
print(compileString("""
var x: integer
program p
  var y: integer
    y := x + 3 + 4
    y := 3 + 4 + x
    y := 3 + x + 4
"""))

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(global $x (mut i32) i32.const 0)
(global $_memsize (mut i32) i32.const 0)
(func $program
(local $y i32)
(local $0 i32)
global.get $x
i32.const 3
i32.add
i32.const 4
i32.add
local.set $y
i32.const 7
global.get $x
i32.add
local.set $y
i32.const 3
global.get $x
i32.add
i32.const 4
i32.add
local.set $y
)
(memory 1)
(start $program)
)


The second assignment adds `7`, but the first and third add `3` and `4` separately. Explain briefly why the P0 compiler generates that code!

Here are the `factor()` and `simpleExpression()` functions of P0.ipynb that deal with addition:
```python
def factor():
    ...
    elif SC.sym == NUMBER:
        x = Const(Int, SC.val); x = CG.genConst(x); getSym()
    ...
        
def simpleExpression():
    ...
    while SC.sym in {PLUS, MINUS, UNION, OR}:
        ...
        if op in {PLUS, MINUS} and x.tp == Int == y.tp:
            if type(x) == Const == type(y): # constant folding
                if op == PLUS: x.val = x.val + y.val
                elif op == MINUS: x.val = x.val - y.val
            ...
        ...
```

In the `factor()` function, number literals such as `3` and `4` are interpreted as constants with type `Const()`. Due to the left-to-right parsing order, the `3 + 4` expression is evaluated before `+ x`, resulting in `7`, as per `if op == PLUS: x.val = x.val + y.val` within the `simpleExpression()` function.


With a restricted range of integers, some programming languages implement strict arithmetic, meaning that overflow is an error, and some languages implement modulo arithmetic, meaning that there is no overflow, but the result "wraps around". Are compilers allowed to optimize the first and third assignments to add `7` with strict arithmetic and modulo arithmetic?

For the first assignment `y := x + 3 + 4`, if the language implements strict arithmetic, the compiler would treat the addition operation as potentially causing overflow, and hence may not optimize it. However, if the compiler can determine that overflow will not occur for the given range of integers, it might optimize the operation into a single constant value, `y := x + 7`.

For the third assignment `y := 3 + x + 4`, since addition is associative, the compiler may choose to rearrange the operands to optimize the computation. In the case of modulo arithmetic, the compiler might optimize it into a single constant value, `y := x + 7` since the addition of `3` and `4` is equivalent to adding `7`. However, in the case of strict arithmetic, if the compiler cannot guarantee that overflow won't occur, it may not perform this optimization.